Este código implementa um classificador Naive Bayes que pode lidar com dados categóricos e contínuos. Temos os seguintes métodos:

__init__(self, categorical_features): Inicializa o modelo com uma lista de índices de recursos que são categóricos. Armazena os índices categóricos e os recursos contínuos em diferentes variáveis.

fit(self, X, y): Ajusta o modelo aos dados de treino X e rótulos y. Calcula a priori de classe, as probabilidades gaussianas para recursos contínuos e as probabilidades categóricas para recursos categóricos para cada classe.

predict(self, X): Faz previsões para o conjunto de dados X. Calcula as probabilidades de cada classe usando as probabilidades gaussianas para recursos contínuos e as probabilidades categóricas para recursos categóricos para cada instância. Retorna as classes com maior probabilidade para cada instância.

In [2]:
import numpy as np

class MixedNaiveBayes:
    def __init__(self, categorical_features):
        self.categorical_features = categorical_features
        self.continuous_features = None

    def fit(self, X, y):
        self.classes, class_counts = np.unique(y, return_counts=True)
        self.class_prior = class_counts / y.shape[0]

        self.continuous_features = np.setdiff1d(np.arange(X.shape[1]), self.categorical_features)

        # Gaussian Naive Bayes
        self.mean = np.zeros((len(self.classes), len(self.continuous_features)))
        self.var = np.zeros((len(self.classes), len(self.continuous_features)))

        # Categorical Naive Bayes
        self.cat_probs = [{} for _ in range(len(self.categorical_features))]

        for idx, c in enumerate(self.classes):
            X_c = X[y == c]

            # Gaussian Naive Bayes
            self.mean[idx, :] = X_c[:, self.continuous_features].mean(axis=0)
            self.var[idx, :] = X_c[:, self.continuous_features].var(axis=0)

            # Categorical Naive Bayes
            for i, cat_feature in enumerate(self.categorical_features):
                unique_cat, cat_counts = np.unique(X_c[:, cat_feature], return_counts=True)
                self.cat_probs[i][c] = dict(zip(unique_cat, cat_counts / X_c.shape[0]))

    def predict(self, X):
        preds = []
        for x in X:
            probs = self.class_prior.copy()
            for idx, c in enumerate(self.classes):
                # Gaussian Naive Bayes
                gaussian_prob = (1 / np.sqrt(2 * np.pi * self.var[idx])) * np.exp(-((x[self.continuous_features] - self.mean[idx])**2 / (2 * self.var[idx])))
                probs[idx] *= np.prod(gaussian_prob)

                # Categorical Naive Bayes
                for i, cat_feature in enumerate(self.categorical_features):
                    probs[idx] *= self.cat_probs[i][c].get(x[cat_feature], 1e-6)

            preds.append(self.classes[np.argmax(probs)])
        return np.array(preds)


**Exemplo de uso**

In [4]:
# Gera dados de exemplo
np.random.seed(42)

# Gera 100 amostras com 3 recursos (2 contínuos e 1 categórico)
X = np.hstack([np.random.randint(0, 2, (100, 1)), np.random.rand(100, 2)])
y = np.random.randint(0, 2, 100)

# Divide os dados em treino e teste
train_idx = np.random.choice(range(100), 80, replace=False)
test_idx = np.setdiff1d(range(100), train_idx)

X_train = X[train_idx]
y_train = y[train_idx]
X_test = X[test_idx]
y_test = y[test_idx]

# Cria e treina o modelo
categorical_features = [0]
clf = MixedNaiveBayes(categorical_features)
clf.fit(X_train, y_train)

# Faz previsões
predictions = clf.predict(X_test)
print("Previsões:", predictions)

# Calcula a acurácia
accuracy = np.mean(predictions == y_test)
print("Acurácia:", accuracy)

Previsões: [0 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
Acurácia: 0.7
